In [96]:
import json
import folium
import requests
import mimetypes
import http.client
import pandas as pd
from folium.plugins import HeatMap
from pandas.io.json import json_normalize

In [97]:
conn=http.client.HTTPSConnection("api.covid19api.com")
payload=""
headers={}
conn.request("GET","/summary",payload,headers)
res=conn.getresponse()
data=res.read().decode('UTF-8')

In [98]:
covid_json=json.loads(data)

In [99]:
covid_json

{'Message': '',
 'Global': {'NewConfirmed': 278775,
  'TotalConfirmed': 30679940,
  'NewDeaths': 5323,
  'TotalDeaths': 955782,
  'NewRecovered': 239074,
  'TotalRecovered': 20918817},
 'Countries': [{'Country': 'Afghanistan',
   'CountryCode': 'AF',
   'Slug': 'afghanistan',
   'NewConfirmed': 36,
   'TotalConfirmed': 38919,
   'NewDeaths': 0,
   'TotalDeaths': 1437,
   'NewRecovered': 0,
   'TotalRecovered': 32576,
   'Date': '2020-09-20T12:25:02Z',
   'Premium': {}},
  {'Country': 'Albania',
   'CountryCode': 'AL',
   'Slug': 'albania',
   'NewConfirmed': 153,
   'TotalConfirmed': 12226,
   'NewDeaths': 5,
   'TotalDeaths': 358,
   'NewRecovered': 57,
   'TotalRecovered': 6888,
   'Date': '2020-09-20T12:25:02Z',
   'Premium': {}},
  {'Country': 'Algeria',
   'CountryCode': 'DZ',
   'Slug': 'algeria',
   'NewConfirmed': 210,
   'TotalConfirmed': 49623,
   'NewDeaths': 6,
   'TotalDeaths': 1665,
   'NewRecovered': 105,
   'TotalRecovered': 34923,
   'Date': '2020-09-20T12:25:02Z',
   

In [100]:
pd.json_normalize(covid_json['Countries'],sep=",")

,Country,CountryCode,Slug,NewConfirmed,TotalConfirmed,NewDeaths,TotalDeaths,NewRecovered,TotalRecovered,Date
0,Afghanistan,AF,afghanistan,36,38919,0,1437,0,32576,2020-09-20T12:25:02Z
1,Albania,AL,albania,153,12226,5,358,57,6888,2020-09-20T12:25:02Z
2,Algeria,DZ,algeria,210,49623,6,1665,105,34923,2020-09-20T12:25:02Z
3,Andorra,AD,andorra,0,1564,0,53,0,1164,2020-09-20T12:25:02Z
4,Angola,AO,angola,53,3901,0,147,2,1445,2020-09-20T12:25:02Z
...,...,...,...,...,...,...,...,...,...,...
183,Viet Nam,VN,vietnam,0,1068,0,35,1,942,2020-09-20T12:25:02Z
184,Western Sahara,EH,western-sahara,0,10,0,1,0,8,2020-09-20T12:25:02Z
185,Yemen,YE,yemen,2,2026,0,585,0,1221,2020-09-20T12:25:02Z
186,Zambia,ZM,zambia,48,14070,1,330,158,13365,2020-09-20T12:25:02Z


In [101]:
df=pd.DataFrame(covid_json['Countries'])

In [102]:
df.head()

,Country,CountryCode,Slug,NewConfirmed,TotalConfirmed,NewDeaths,TotalDeaths,NewRecovered,TotalRecovered,Date,Premium
0,Afghanistan,AF,afghanistan,36,38919,0,1437,0,32576,2020-09-20T12:25:02Z,{}
1,Albania,AL,albania,153,12226,5,358,57,6888,2020-09-20T12:25:02Z,{}
2,Algeria,DZ,algeria,210,49623,6,1665,105,34923,2020-09-20T12:25:02Z,{}
3,Andorra,AD,andorra,0,1564,0,53,0,1164,2020-09-20T12:25:02Z,{}
4,Angola,AO,angola,53,3901,0,147,2,1445,2020-09-20T12:25:02Z,{}


In [103]:
df.drop(['CountryCode','Slug','Date','Premium'],axis=1)

,Country,NewConfirmed,TotalConfirmed,NewDeaths,TotalDeaths,NewRecovered,TotalRecovered
0,Afghanistan,36,38919,0,1437,0,32576
1,Albania,153,12226,5,358,57,6888
2,Algeria,210,49623,6,1665,105,34923
3,Andorra,0,1564,0,53,0,1164
4,Angola,53,3901,0,147,2,1445
...,...,...,...,...,...,...,...
183,Viet Nam,0,1068,0,35,1,942
184,Western Sahara,0,10,0,1,0,8
185,Yemen,2,2026,0,585,0,1221
186,Zambia,48,14070,1,330,158,13365


In [104]:
m=folium.Map(tiles='Stamen Terrain',min_zoom=1.5)
m

In [105]:
url='https://raw.githubusercontent.com/python-visualization/folium/master/examples/data'
country_shapes=f'{url}/world-countries.json'

In [106]:
folium.Choropleth(
    geo_data=country_shapes,
    min_zoom=2,
    name='Covid-19',
    data=df,
    key_on='feature.properties.name',
    columns=['Country','TotalConfirmed'],
    fill_color='OrRd',
    nan_fill_color='black',
    legend_name='Total Confirmed Covid Cases',
).add_to(m)
m

In [107]:
df.update(df['TotalConfirmed'].map('Total Confirmed: {}'.format))
df.update(df['TotalRecovered'].map('Total Recovered: {}'.format))

In [108]:
coordinates=pd.read_csv("https://raw.githubusercontent.com/VinitaSilaparasetty/covid-map/master/country-coordinates-world.csv")
coordinates

,latitude,longitude,Country
0,33.939110,67.709953,Afghanistan
1,41.153332,20.168331,Albania
2,28.033886,1.659626,Algeria
3,-14.270972,-170.132217,American Samoa
4,42.546245,1.601554,Andorra
...,...,...,...
239,-13.768752,-177.156097,Wallis and Futuna
240,24.215527,-12.885834,Western Sahara
241,15.552727,48.516388,Yemen
242,-13.133897,27.849332,Zambia


In [109]:
covid_df=pd.merge(df,coordinates,on='Country')
covid_df

,Country,CountryCode,Slug,NewConfirmed,TotalConfirmed,NewDeaths,TotalDeaths,NewRecovered,TotalRecovered,Date,Premium,latitude,longitude
0,Afghanistan,AF,afghanistan,36,Total Confirmed: 38919,0,1437,0,Total Recovered: 32576,2020-09-20T12:25:02Z,{},33.939110,67.709953
1,Albania,AL,albania,153,Total Confirmed: 12226,5,358,57,Total Recovered: 6888,2020-09-20T12:25:02Z,{},41.153332,20.168331
2,Algeria,DZ,algeria,210,Total Confirmed: 49623,6,1665,105,Total Recovered: 34923,2020-09-20T12:25:02Z,{},28.033886,1.659626
3,Andorra,AD,andorra,0,Total Confirmed: 1564,0,53,0,Total Recovered: 1164,2020-09-20T12:25:02Z,{},42.546245,1.601554
4,Angola,AO,angola,53,Total Confirmed: 3901,0,147,2,Total Recovered: 1445,2020-09-20T12:25:02Z,{},-11.202692,17.873887
...,...,...,...,...,...,...,...,...,...,...,...,...,...
162,Uzbekistan,UZ,uzbekistan,739,Total Confirmed: 50992,8,427,744,Total Recovered: 47271,2020-09-20T12:25:02Z,{},41.377491,64.585262
163,Western Sahara,EH,western-sahara,0,Total Confirmed: 10,0,1,0,Total Recovered: 8,2020-09-20T12:25:02Z,{},24.215527,-12.885834
164,Yemen,YE,yemen,2,Total Confirmed: 2026,0,585,0,Total Recovered: 1221,2020-09-20T12:25:02Z,{},15.552727,48.516388
165,Zambia,ZM,zambia,48,Total Confirmed: 14070,1,330,158,Total Recovered: 13365,2020-09-20T12:25:02Z,{},-13.133897,27.849332


In [110]:
def plotDot(point):
    folium.CircleMarker(location=[point.latitude,point.longitude],
                         radius=5,
                         weight=2,
                         popup=[point.Country,point.TotalConfirmed,point.TotalRecovered],
                         fill_Color='#000000').add_to(m)

In [111]:
covid_df.apply(plotDot,axis=1)
m.fit_bounds(m.get_bounds())
m

In [112]:
m1=folium.Map(tiles='StamenToner',min_zoom=2)
m1

In [113]:
deaths=covid_df['TotalDeaths'].astype(float)
lat=covid_df['latitude'].astype(float)
long=covid_df['longitude'].astype(float)
m1.add_child(HeatMap(zip(lat,long,deaths),radius=0))

In [116]:
def plotDot1(point):
    folium.CircleMarker(location=[point.latitude,point.longitude],
                         radius=5,
                         weight=2,
                         popup=[point.Country,point.TotalDeaths],
                         fill_Color='#000000').add_to(m)

In [117]:
covid_df.apply(plotDot1,axis=1)
m1.fit_bounds(m1.get_bounds())
m1

AttributeError: 'Series' object has no attribute 'latitude'